In [65]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
data_path = 'marketing_campaign.csv'

campaign = pd.read_csv(data_path)

In [67]:
campaign.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,88,3,8,10,4,7,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,6,2,1,1,2,5,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,42,1,8,2,10,4,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,...,5,2,2,0,4,6,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,15,5,5,3,6,5,0,3,11,0


In [68]:
dummy_fields = ['Year_Birth', 'Education', 'Marital_Status']
for each in dummy_fields:
    dummies = pd.get_dummies(rides[each], prefix=each, drop_first=False)
    campaign = pd.concat([rides, dummies], axis=1)

fields_to_drop = ['ID', 'Dt_Customer', 'Year_Birth', 'Education', 'Marital_Status']
data = campaign.drop(fields_to_drop, axis=1)
data.head()


,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,...,Z_Revenue,Response,Marital_Status_Absurd,Marital_Status_Alone,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Together,Marital_Status_Widow,Marital_Status_YOLO
0,58138.0,0,0,58,635,88,546,172,88,88,...,11,1,0,0,0,0,1,0,0,0
1,46344.0,1,1,38,11,1,6,2,1,6,...,11,0,0,0,0,0,1,0,0,0
2,71613.0,0,0,26,426,49,127,111,21,42,...,11,0,0,0,0,0,0,1,0,0
3,26646.0,1,0,26,11,4,20,10,3,5,...,11,0,0,0,0,0,0,1,0,0
4,58293.0,1,0,94,173,43,118,46,27,15,...,11,0,0,0,0,1,0,0,0,0


In [69]:
print(data.columns.values)

['Income' 'Kidhome' 'Teenhome' 'Recency' 'MntWines' 'MntFruits'
 'MntMeatProducts' 'MntFishProducts' 'MntSweetProducts' 'MntGoldProds'
 'NumDealsPurchases' 'NumWebPurchases' 'NumCatalogPurchases'
 'NumStorePurchases' 'NumWebVisitsMonth' 'AcceptedCmp3' 'AcceptedCmp4'
 'AcceptedCmp5' 'AcceptedCmp1' 'AcceptedCmp2' 'Complain' 'Z_CostContact'
 'Z_Revenue' 'Response' 'Marital_Status_Absurd' 'Marital_Status_Alone'
 'Marital_Status_Divorced' 'Marital_Status_Married'
 'Marital_Status_Single' 'Marital_Status_Together' 'Marital_Status_Widow'
 'Marital_Status_YOLO']


### Scaling target variables

In [70]:
quant_features = ['Income', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts','MntSweetProducts','MntGoldProds','NumDealsPurchases','NumWebPurchases','NumCatalogPurchases','NumStorePurchases','NumWebVisitsMonth','Complain','Z_CostContact','Z_Revenue']
# Store scalings in a dictionary so we can convert back later
scaled_features = {}
for each in quant_features:
    mean, std = data[each].mean(), data[each].std()
    scaled_features[each] = [mean, std]
    data.loc[:, each] = (data[each] - mean)/std


### training, validation and test data

In [71]:

test_data = data[-200:]

# Now remove the test data from the data set 
data = data[:-200]

# Separate the data into features and targets
target_fields = ['Response']
features, targets = data.drop(target_fields, axis=1), data[target_fields]
test_features, test_targets = test_data.drop(target_fields, axis=1), test_data[target_fields]

In [72]:
train_features, train_targets = features[:-300], targets[:-300]
val_features, val_targets = features[-300:], targets[-300:]

In [73]:
type(train_features)

pandas.core.frame.DataFrame

In [74]:
import torch

#Check if your system supports CUDA
gpu_available = torch.cuda.is_available() # is_available returns a bool as true or flase

if gpu_available:
    print('Training on GPU')
else:
    print('Training on CPU')

Training on GPU


In [75]:
train_features_t=torch.tensor(train_features.values)
train_targets_t = torch.tensor(train_targets.values)
val_features_t = torch.tensor(val_features.values)
val_targets_t = torch.tensor(val_targets.values)

In [76]:
train_features_t.size()

torch.Size([1740, 31])

In [77]:
train_targets_t.size()

torch.Size([1740, 1])

In [78]:
train_features_loader = torch.utils.data.DataLoader(train_features_t)
train_targets_loader = torch.utils.data.DataLoader(train_targets_t)
val_features_loader = torch.utils.data.DataLoader(val_features_t)
val_targets_loader = torch.utils.data.DataLoader(val_targets_t)

### Neural Network Architecture


In [79]:
import torch.nn as nn
import torch.nn.functional as F

# CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(31,128)
        self.fc2 = nn.Linear(128,64)
        self.fc3 = nn.Linear(64,1)
        
    def forward(self, x):
        ## Define forward behavior
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x), dim=1)
        return x

# instantiate the CNN
model = Net()
print(model)

# move tensors to GPU if CUDA is available
if gpu_available:
    model.cuda()

Net(
  (fc1): Linear(in_features=31, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
)


In [80]:
import torch.optim as optim

criterion = nn.MSELoss()

# using Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Training and Validation 1

In [82]:
# TODO: Train the network here
epochs = 5

for e in range(epochs):
    running_loss = 0
    for features, labels in enumerate(zip(train_features_loader,train_targets_loader)):
        log_ps = model(features)
        loss = criterion(log_ps, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(train_features_loader)}")

AttributeError: 'int' object has no attribute 'dim'

### Training and Validation 2

In [61]:
# TODO: Train the network here
epochs = 5

for e in range(epochs):
    running_loss = 0
    for features in enumerate(train_features_loader):
        log_ps = model(features)

AttributeError: 'tuple' object has no attribute 'dim'